In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
stop_words=list(stopwords.words('english'))
from sklearn import svm
import pickle
from sklearn.model_selection import train_test_split
from sklearn import svm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [108]:
class myPipeline:

    def __init__(self, df,isTrain=True):
        self.df = df
        self.df.dropna(inplace=True)
        self.isTrain=isTrain
        self.labelencoder= LabelEncoder()
        self.count_vect=CountVectorizer()
        self.tfidf=TfidfTransformer()
        self.model=svm.LinearSVC()
        
    def text_preprocessing(self,txt):
        txt=txt.replace("\n","").replace('"',"").lower()
        txt = re.sub('[^a-zA-Z0-9 \n]', ' ', txt)
        txt=re.sub('\s+',' ',txt)
        return txt

    def processing_steps(self):
        #label encoding the target
        self.df['Package'] = self.labelencoder.fit_transform(self.df['Package'])
        # processing the text columns
        self.df["Sample"]=self.df["Sample"].apply(self.text_preprocessing)
        self.df["Medical_Description"]=self.df["Medical_Description"].apply(self.text_preprocessing)
        #removing stop words from feature columns
        self.df['Sample'] = self.df['Sample'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
        self.df['Medical_Description'] =  self.df['Medical_Description'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
        #combining the features
        self.df["Processed_text"]=self.df["Sample"]+" "+self.df["Medical_Description"]
        #self.df["Processed_text"]=self.df["Processed_text"].apply(lambda x:word_tokenize(str(x)))
        self.model_prediction()
        return self.df
        
    
    def model_prediction(self):
        if self.isTrain:
            X_train=self.count_vect.fit_transform(self.df.Processed_text)
            X_train_tfidf=self.tfidf.fit_transform(X_train)
            X_train,X_test,y_train,y_test=train_test_split(X_train_tfidf,train.Package,test_size=0.25,random_state=42)
            self.model.fit(X_train_tfidf,train.Package)
            pickle.dump(self.count_vect.vocabulary_,open("count_vector.pkl","wb"))
            pickle.dump(self.tfidf,open("tfidf_vector.pkl","wb"))
            pickle.dump( self.model,open("svm.pkl","wb"))
            predicted=self.model.predict(X_test)
            
            result_svm=pd.DataFrame({"true labels":y_test,"predicted":predicted})
            f1_score=self.evaluation(y_test,predicted)
            print("f1 score is",f1_score)
#             for predicted_item,result in zip(predicted,y_test):
#                 print(predicted_item,"-",result)
        else:
            #load from the picke file
            load_count_vec=CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl","rb")))
            load_tfidf_vec=pickle.load(open("tfidf_vector.pkl","rb"))
            model=pickle.load(open("svm.pkl","rb"))
            X_count=load_count_vec.transform(self.df.Processed_text)
            X_tfidf=load_tfidf_vec.transform(X_count)
            predicted= model.predict(X_tfidf)
            self.df["Package"]=self.labelencoder.inverse_transform(self.df.Package.values)
            self.df["Predictions"]=self.labelencoder.inverse_transform(predicted)
         
    def evaluation(self,y_test,predicted):
        return np.round(f1_score(y_test, predicted, average='micro'),2)
        

In [109]:
train=pd.read_csv("Case_Study_Data_Train.csv")
test=pd.read_csv("Case_Study_Data_Test.csv")


In [110]:
pipeline = myPipeline(train)  
result=pipeline.processing_steps()

f1 score is 0.91


In [111]:
pipelinetest = myPipeline(test,False)  
result=pipelinetest.processing_steps()
result.head()

,Package,Sample,Medical_Description,Processed_text,Predictions
0,Ophthalmology,browlift blepharoplasty rhytidectomy,preoperative diagnoses 1 eyebrow ptosis 2 derm...,browlift blepharoplasty rhytidectomy preoperat...,Ophthalmology
1,Orthopedic,lumbar discogram,preoperative diagnosis low back pain postopera...,lumbar discogram preoperative diagnosis low ba...,Neurology
2,Cardiovascular / Pulmonary,heart catheterization angiography 1,clinical indication normal stress test procedu...,heart catheterization angiography 1 clinical i...,Cardiovascular / Pulmonary
3,Ophthalmology,ophthalmology letter 1,xyz abc dob mm dd yyyy dear dr xyz thank refer...,ophthalmology letter 1 xyz abc dob mm dd yyyy ...,Ophthalmology
4,Neurosurgery,scott cannula,procedure placement scott cannula right latera...,scott cannula procedure placement scott cannul...,Neurosurgery
